# Task 2

Perform classification of protein types based on codon usage in SARS-CoV2.

- use the [data](alas.matf.bg.ac.rs) containing coding sequences and protein types, prepared from [NCBI Virus](https://www.ncbi.nlm.nih.gov/labs/virus/vssi/#/virus?SeqType_s=Nucleotide&VirusLineage_ss=SARS-CoV-2,%20taxid:2697049).

- use standard codon table from [The Genetic Codes](https://www.ncbi.nlm.nih.gov/Taxonomy/Utils/wprintgc.cgi).
Za SARS2 koristiti standardni kod (transl_table=1)

In [ ]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import dtreeviz
import dalex as dx

from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import ComplementNB
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import average_precision_score, precision_recall_curve as prec_rec_crve, balanced_accuracy_score, classification_report

from mlxtend.evaluate import confusion_matrix as conf_matrx, RandomHoldoutSplit
from mlxtend.plotting import plot_confusion_matrix

from openTSNE import TSNE

from annoy import AnnoyIndex

from xgboost import XGBClassifier

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Recall

from scikeras.wrappers import KerasClassifier

from sklearn_genetic import ExponentialAdapter, GASearchCV
from sklearn_genetic.callbacks import DeltaThreshold
from sklearn_genetic.space import Categorical, Integer, Continuous

from yellowbrick.target import class_balance
from yellowbrick.target.feature_correlation import feature_correlation
from yellowbrick.features.pcoords import parallel_coordinates
from yellowbrick.features.radviz import radviz
from yellowbrick.classifier import confusion_matrix, precision_recall_curve
from yellowbrick.model_selection import feature_importances
from yellowbrick.contrib.wrapper import classifier

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=PendingDeprecationWarning)

stop = 5
start = 7

tous = 11
pops = 31
seed = 196883

sample = 4224
maxiter = np.iinfo(np.int32).max

t_stop = 1e-4
t_sample = 0.2

a_rate = 0.1
m_prob = 0.2
c_prob = 0.8

n_features = 7

np.random.seed(seed)

## Priprema podataka

Učitavamo podatke sa jedinstvenim kodirajućim sekvencama proteina SARS-CoV-2.

In [ ]:
df = pd.read_csv('samo_jedinstvene_kodirajuce_sekvence.csv', header=None, on_bad_lines='warn')

In [ ]:
df.shape

In [ ]:
df.head()

Izdvajamo kolonu sa kodirajućim sekvencama i kolonu sa oznakama tipova proteina.

In [ ]:
df = df[[14, 12]]
df.shape

### Čišćenje podataka

Proveravamo da li su kodirajuće sekvence odgovarajuće dužine.
One koje nisu, odbacujemo.

In [ ]:
length = np.array([len(a) for a in df[14]])
valid = (length > 8) & (length % 3 == 0)

df = df[valid]
df.shape

Proveravamo da li se kodirajuće sekvence sastoje samo od slova A, T, C i G.
Sekvence koje ne ispunjavaju ovaj uslov odbacujemo.

In [ ]:
valid = df[14].map(lambda a: set(a).issubset(set('ATCG')))

df = df[valid]
df.shape

Proveravamo koliko instanci imamo u svakoj klasi.
Odnosno koliko sekvenci imamo za svaki tip proteina.
Klase koje sadrže 64 instanci i manje odbacujemo zajedno sa instancom.

In [ ]:
proteins = df.groupby([12])[12].count()
valid = np.array([proteins[a] for a in df[12]]) > 64

df = df[valid]
df.shape

Nakon čišćenja podataka, od jedinstvenih 889737 kodirajućih ostalo nam je 869858 instanci.
Ukupno 19879 instanci je odbačeno kao nevalidno, odnosno oko $2.23\%$ skupa jedinstvenih kodirajućih sekvenci.

### Izračunavanje upotrebe kodona

Računamo koliko često neki kodon kodira svoju amino-kiselinu u svakoj kodirajućoj sekvenci.

Potrebna nam je standardna tabela kodona.

In [ ]:
standard_codon_table = {
    'TTT': 'F', 'TCT': 'S', 'TAT': 'Y', 'TGT': 'C',
    'TTC': 'F', 'TCC': 'S', 'TAC': 'Y', 'TGC': 'C',
    'TTA': 'L', 'TCA': 'S', 'TAA': 'O', 'TGA': 'O',
    'TTG': 'L', 'TCG': 'S', 'TAG': 'O', 'TGG': 'W',

    'CTT': 'L', 'CCT': 'P', 'CAT': 'H', 'CGT': 'R',
    'CTC': 'L', 'CCC': 'P', 'CAC': 'H', 'CGC': 'R',
    'CTA': 'L', 'CCA': 'P', 'CAA': 'Q', 'CGA': 'R',
    'CTG': 'L', 'CCG': 'P', 'CAG': 'Q', 'CGG': 'R',

    'ATT': 'I', 'ACT': 'T', 'AAT': 'N', 'AGT': 'S',
    'ATC': 'I', 'ACC': 'T', 'AAC': 'N', 'AGC': 'S',
    'ATA': 'I', 'ACA': 'T', 'AAA': 'K', 'AGA': 'R',
    'ATG': 'M', 'ACG': 'T', 'AAG': 'K', 'AGG': 'R',

    'GTT': 'V', 'GCT': 'A', 'GAT': 'D', 'GGT': 'G',
    'GTC': 'V', 'GCC': 'A', 'GAC': 'D', 'GGC': 'G',
    'GTA': 'V', 'GCA': 'A', 'GAA': 'E', 'GGA': 'G',
    'GTG': 'V', 'GCG': 'A', 'GAG': 'E', 'GGG': 'G'
}

Potrebna nam je i inverzna tabela kodona.

In [ ]:
inverse_codon_table = {
    'F': ('TTT', 'TTC'),
    'L': ('CTT', 'CTC', 'CTA', 'CTG', 'TTA', 'TTG'),
    'I': ('ATT', 'ATC', 'ATA'),
    'M': ('ATG', ),
    'V': ('GTT', 'GTC', 'GTA', 'GTG'),
    'S': ('TCT', 'TCC', 'TCA', 'TCG', 'AGT', 'AGC'),
    'P': ('CCT', 'CCC', 'CCA', 'CCG'),
    'T': ('ACT', 'ACC', 'ACA', 'ACG'),
    'A': ('GCT', 'GCC', 'GCA', 'GCG'),
    'Y': ('TAT', 'TAC'),
    'O': ('TAA', 'TGA', 'TAG'),
    'H': ('CAT', 'CAC'),
    'Q': ('CAA', 'CAG'),
    'N': ('AAT', 'AAC'),
    'K': ('AAA', 'AAG'),
    'D': ('GAT', 'GAC'),
    'E': ('GAA', 'GAG'),
    'C': ('TGT', 'TGC'),
    'W': ('TGG', ),
    'R': ('CGT', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'),
    'G': ('GGT', 'GGC', 'GGA', 'GGG')
}

Za svaku sekvencu brojimo koliko sadrži kodona.

In [ ]:
def count_codons(seq):
    seq = seq[14]
    
    codon_counts = {a: 0 for a in standard_codon_table.keys()}
    
    for i in range(0, len(seq) - 2, 3):
        codon = seq[i:i+3]
        codon_counts[codon] += 1
    
    return codon_counts

codon_counts = df[[14]].apply(count_codons, axis=1, result_type='expand').to_dict('list')
codon_counts = {key: np.array(val, dtype='float64') for key, val in codon_counts.items()}

Za prebrojavanje amino-kiselina u prevedenim sekvencama, nećemo eksplicitno prevoditi sekvence i brojati.
Iskoristićemo činjenicu da smo već izbrojali kodone u kodirajućim sekvencama.
Sada ostaje da za svaku amino-kiselinu saberemo broj kodona koji ih kodiraju, za svaku sekvencu.

In [ ]:
length = df.shape[0]

amino_counts = {a: np.zeros(length) for a in inverse_codon_table.keys()}

for amino, codons in inverse_codon_table.items():
    for codon in codons:
        amino_counts[amino] += codon_counts[codon]

Neka se kodon *X* pojavljuje *x* puta u nekoj sekvenci i kodira amino-kiselinu *Y*.
U prevedenoj sekvenci, nakon transkripcije i translacije, amino-kiselina *Y* se pojavljuje *y* puta.
Vrednost
$$
f(X) = \frac{x}{y}
$$
predstavlja upotrebu kodona *X*.

Postoje sekvence koje ne kodiraju sve amino-kiseline, što prilikom primene formule dovodi do deljenja nulom i dodele *NaN* vrednosti.
Zamenićemo *NaN* vrednosti nulom.

In [ ]:
for codon in standard_codon_table.keys():
    codon_counts[codon] /= amino_counts[standard_codon_table[codon]]
    np.nan_to_num(codon_counts[codon], copy=False, nan=0, posinf=0, neginf=0) # ako delimo nulom

### Završna priprema podataka i podela na skupove

Rečnik sa vrednostima upotrebe kodona prevodimo u pogodniji format za rad sa tabelarnim podacima.
Odbacujemo podatke o upotrebi stop kodona.

In [ ]:
X = pd.DataFrame(codon_counts)
X.drop(['TAA', 'TGA', 'TAG', 'ATG'], axis=1, inplace=True)

y = df[12]
y.name = 'Protein'

df = X.join(y, validate='one_to_one')

cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]

# df.to_csv('proteini_ucestalost_kodona.csv', index=False)
df.head()

Pravimo enkoder koji kateogričkim ciljnim podacima dodeljuje celobrojne vrednosti.

In [ ]:
enc = LabelEncoder()
_ = enc.fit(y.unique())

In [ ]:
ohe = LabelBinarizer()
_ = ohe.fit(y.unique())

Konačno, delimo podatke na podatke za test i trening skupove.
Test skup će se sadržati 20% podataka, a ostatak će sadržati trening skup.
Pravimo stratifikovanu podelu, kako bi sve klase bile jednako zastupljene u test i trening skupu.
Dalje u analizi ćemo se fokusirati na trening skup.
Test skup ćemo koristiti isključivo za proveru modela.
U svim ostalim situacijama ponašamo se kao da test skup ne postoji.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed, stratify=y)

A small training data sample that will be used for certain visualization techniques.

In [ ]:
train_sample = X_train.join(y_train, validate='one_to_one').groupby(['Protein']).sample(8, random_state=seed).reset_index(drop=True)

X_train_sample = train_sample.drop(['Protein'], axis=1)
y_train_sample = train_sample['Protein']

Long form of the small sampled training data set.

In [ ]:
shape = train_sample.shape[0] * (train_sample.shape[1] - 1)

train_sample_long = {
    'index': np.empty(shape, np.int32),
    'codon': [None for _ in range(shape)],
    'usage': np.empty(shape, np.float64),
    'protein': [None for _ in range(shape)]
}

j = 0
for i, s in X_train_sample.iterrows():
    for codon, freq in s.items():
        train_sample_long['index'][j] = i
        train_sample_long['codon'][j] = codon
        train_sample_long['usage'][j] = freq
        train_sample_long['protein'][j] = y_train_sample.iloc[i]
        j += 1

train_sample_long = pd.DataFrame(train_sample_long)

## Data vizualization

The classes are very unbalanced.
About $79\%$ instances belong to *ORF1ab* i *ORF1a*.
More than $95\%$ of instances are in five, out of total twelve classes.

In [ ]:
_ = plt.figure(figsize=(16, 9))

_ = class_balance(y_train,
                  labels=[a.split()[0] for a in enc.classes_])

### Stats for target and features

We will look at correlations between target variable and the features to see if there are any variables that stand out.

In [ ]:
_ = plt.figure(figsize=(16, 9))

_ = feature_correlation(X=X_train,
                        y=y_train,
                        method='mutual_info-classification')

### Parallel Coordinates

Certainly this is an interesting looking plot.
Its usefulness leaves some doubts, though.

In [ ]:
_ = plt.figure(figsize=(16, 9))

_ = parallel_coordinates(X=X_train_sample,
                         y=y_train_sample,
                         alpha=0.6,
                         normalize='standard')

### Radar Chart

Just like parallel coordinates, but a bit different.
Abstract art.
Although there are some overlappings, classes appear to be separable.

In [ ]:
_ = plt.figure(figsize=(16, 9))

_ = radviz(X=X_train,
       y=enc.transform(y_train),
       classes=enc.classes_,
       alpha=0.6,
       colormap='jet')

### t-SNE

T-distributed stochastic neighbor embedding, shorter t-SNE, is nonlinear algorithm for dimensionality reduction.
Due to its stochastic nature, this algorithm is suitable only for data visualization.

In [ ]:
tsne = TSNE(n_components=2,
            perplexity=48,
            exaggeration=4,
            n_iter=8192,
            early_exaggeration_iter=1024,
            n_jobs=-1, metric='manhattan',
            learning_rate='auto',
            initialization='random',
            random_state=seed)

X_train_tsne = tsne.fit(X_train.to_numpy())

Classes separate without overlapping, which is a good result, and promisses good classification.

In [ ]:
_ = plt.figure(figsize=(16, 9))

ax = sns.scatterplot(x=X_train_tsne[:, 0],
                     y=X_train_tsne[:, 1],
                     hue=y_train,
                     palette='jet')
# ax.set(xlabel='', ylabel='')

plt.show()

## Training and evaluation

For parameter tuning we will use cross-validation.
For cross-validation *balanced accuracy* metric will be used.

During the evaluation we will observe model performance on training and test datasets.
Aside from classification report with metrics, we will look at confusion matrices and precision-recall curves.
Finally, we will take a look at feature importances.

First, let us write some utilities.

In [ ]:
protein_labels = enc.classes_
labels2display = np.array([a.split(' ', 1)[0] for a in protein_labels])

In [ ]:
mutation_adapter = ExponentialAdapter(initial_value=c_prob,
                                      end_value=m_prob,
                                      adaptive_rate=a_rate)

In [ ]:
crossover_adapter = ExponentialAdapter(initial_value=m_prob,
                                       end_value=c_prob,
                                       adaptive_rate=a_rate)

In [ ]:
stopping_condition = DeltaThreshold(threshold=t_stop,
                                    generations=stop,
                                    metric='fitness_max')

In [ ]:
early_stopping = EarlyStopping(monitor='loss',
                               min_delta=t_stop,
                               patience=stop,
                               start_from_epoch=start,
                               restore_best_weights=True)

In [ ]:
cross_validation_generator = RandomHoldoutSplit(valid_size=0.2,
                                                random_seed=seed,
                                                stratify=True)

In [ ]:
def display_best_parameters(estimator):
    print(', '.join([f'{a}: {b}' for a, b in estimator.best_params_.items()]))

### Complement Naive Bayes Classifier

Parameters for cross-validation.

In [ ]:
params = {'alpha': Continuous(1e-3, 1e3, random_state=seed)}

Applying genetic search cross-validation algorithm.

In [ ]:
clf = GASearchCV(ComplementNB(),
                 param_grid=params,
                 scoring='balanced_accuracy',
                 population_size=pops,
                 generations=maxiter,
                 tournament_size=tous,
                 elitism=True,
                 crossover_probability=crossover_adapter,
                 mutation_probability=mutation_adapter,
                 cv=cross_validation_generator,
                 algorithm='eaSimple',
                 n_jobs=-1,
                 verbose=False)

_ = clf.fit(X_train,
            y_train,
            callbacks=stopping_condition)

Parameters of the best model.

In [ ]:
display_best_parameters(clf)

Classification report for training data.

In [ ]:
print(classification_report(y_true=y_train,
                            y_pred=clf.predict(X_train),
                            digits=5,
                            zero_division=0.0))

Confusion matrix for training data.

In [ ]:
_ = plt.figure(figsize=(16, 9))

_ = confusion_matrix(clf.best_estimator_,
                     X_train,
                     y_train,
                     classes=labels2display,
                     cmap='Greens',
                     percent=True,
                     is_fitted=True)

Precision-recall curves for training data.

In [ ]:
_ = plt.figure(figsize=(16, 9))

_ = precision_recall_curve(clf.best_estimator_,
                           X_train,
                           y_train,
                           per_class=True)

Classification report for test data.

In [ ]:
print(classification_report(y_true=y_test,
                            y_pred=clf.predict(X_test),
                            digits=5,
                            zero_division=0.0))

Confusion matrix for test data.

In [ ]:
_ = plt.figure(figsize=(16, 9))

_ = confusion_matrix(clf.best_estimator_,
                     X_test,
                     y_test,
                     classes=labels2display,
                     cmap='Blues',
                     percent=True,
                     is_fitted=True)

Precision-recall curves for test data.

In [ ]:
_ = plt.figure(figsize=(16, 9))

_ = precision_recall_curve(clf.best_estimator_,
                           X_test,
                           y_test,
                           per_class=True)

Važnost atributa za test skup.

In [ ]:
explainer = dx.Explainer(clf.best_estimator_,
                         X_train_sample,
                         enc.transform(y_train_sample),
                         predict_function=lambda a, b: enc.transform(a.predict(b)),
                         model_type='classification',
                         label='Variable importance measures for ComplementNB',
                         verbose=False)

In [ ]:
mp_clf = explainer.model_parts(N=sample,
                               B=start,
                               type='shap_wrapper',
                               shap_explainer_type='KernelExplainer',
                               loss_function=lambda a, b: 1-balanced_accuracy_score(a, b),
                               processes=-1,
                               random_state=seed,
                               data=X_test,
                               silent=True)

In [ ]:
mp_clf.plot(max_display=n_features,
            plot_size=(16, 9),
            title='Variable importance measures for ComplementNB')

In [ ]:
_ = plt.figure(figsize=(16, 9))

mp_clf.plot(plot_type='bar',
            max_display=n_features,
            plot_size=(16, 9),
            title='Variable importance measures for ComplementNB')

### Decision Tree

Parameters for cross-validation.

In [ ]:
params = {'criterion': Categorical(['gini', 'entropy'], random_state=seed),
          'max_depth': Integer(4, 32, random_state=seed),
          'min_samples_split': Integer(128, 256, random_state=seed),
          'min_samples_leaf': Integer(32, 64, random_state=seed),
          'min_impurity_decrease': Continuous(0, 1, random_state=seed),
          'ccp_alpha': Continuous(0, 2, random_state=seed)}

Applying genetic search cross-validation algorithm.

In [ ]:
clf = GASearchCV(DecisionTreeClassifier(class_weight='balanced',
                                        random_state=seed),
                 param_grid=params,
                 scoring='balanced_accuracy',
                 population_size=pops,
                 generations=maxiter,
                 tournament_size=tous,
                 elitism=True,
                 crossover_probability=crossover_adapter,
                 mutation_probability=mutation_adapter,
                 cv=cross_validation_generator,
                 algorithm='eaSimple',
                 n_jobs=-1,
                 verbose=False)

_ = clf.fit(X_train,
            enc.transform(y_train),
            callbacks=stopping_condition)

Parameters of the best model.

In [ ]:
display_best_parameters(clf)

Classification report for training data.

In [ ]:
print(classification_report(y_true=y_train,
                            y_pred=enc.inverse_transform(clf.predict(X_train)),
                            digits=5,
                            zero_division=0.0))

Confusion matrix for training data.

In [ ]:
_ = plt.figure(figsize=(16, 9))

_ = confusion_matrix(clf.best_estimator_,
                     X_train,
                     enc.transform(y_train),
                     classes=labels2display,
                     cmap='Greens',
                     percent=True,
                     is_fitted=True)

Precision-recall curves for training data.

In [ ]:
_ = plt.figure(figsize=(16, 9))

_ = precision_recall_curve(clf.best_estimator_,
                           X_train,
                           y_train,
                           encoder=enc,
                           per_class=True)

Classification report for test data.

In [ ]:
print(classification_report(y_true=y_test,
                            y_pred=enc.inverse_transform(clf.predict(X_test)),
                            digits=5,
                            zero_division=0.0))

Confusion matrix for test data.

In [ ]:
_ = plt.figure(figsize=(16, 9))

_ = confusion_matrix(clf.best_estimator_,
                     X_test,
                     enc.transform(y_test),
                     classes=labels2display,
                     cmap='Blues',
                     percent=True,
                     is_fitted=True)

Precision-recall curves for test data.

In [ ]:
_ = plt.figure(figsize=(16, 9))

_ = precision_recall_curve(clf.best_estimator_,
                           X_test,
                           y_test,
                           encoder=enc,
                           per_class=True)

Graphical view of the decision tree classifier.

In [ ]:
viz_model = dtreeviz.model(clf.best_estimator_,
                           X_train=X_train.values,
                           y_train=enc.transform(y_train),
                           feature_names=X_train.columns.to_list(),
                           target_name='Proteins',
                           class_names=list(enc.inverse_transform(clf.best_estimator_.classes_)))

In [ ]:
viz_model.view(scale=0.75,
               leaftype='barh')

Variable importances.

In [ ]:
explainer = dx.Explainer(clf.best_estimator_,
                         X_train_sample,
                         enc.transform(y_train_sample),
                         model_type='classification',
                         label='Variable importance measures for DecisionTreeClassifier',
                         verbose=False)

In [ ]:
mp_clf = explainer.model_parts(N=sample,
                               B=start,
                               type='shap_wrapper',
                               shap_explainer_type='KernelExplainer',
                               loss_function=lambda a, b: 1-balanced_accuracy_score(a, b),
                               processes=-1,
                               random_state=seed,
                               data=X_test,
                               silent=True)

In [ ]:
mp_clf.plot(max_display=n_features,
            plot_size=(16, 9),
            title='Variable importance measures for DecisionTreeClassifier')

In [ ]:
mp_clf.plot(plot_type='bar',
            max_display=n_features,
            plot_size=(16, 9),
            title='Variable importance measures for DecisionTreeClassifier')

### Random Forest

Cross-validation parameters.

In [ ]:
params = {'n_estimators': Integer(32, 256, random_state=seed),
           'criterion': Categorical(['gini', 'entropy'], random_state=seed),
           'max_features': Continuous(0, 1, random_state=seed),
           'max_samples': Continuous(0, 1, random_state=seed)}

Applying genetic search cross-validation algorithm.

In [ ]:
clf = GASearchCV(RandomForestClassifier(n_jobs=-1,
                                        random_state=seed),
                 param_grid=params,
                 scoring='balanced_accuracy',
                 population_size=pops,
                 generations=maxiter,
                 tournament_size=tous,
                 elitism=True,
                 crossover_probability=crossover_adapter,
                 mutation_probability=mutation_adapter,
                 cv=cross_validation_generator,
                 algorithm='eaSimple',
                 n_jobs=2,
                 verbose=False)

_ = clf.fit(X_train,
            y_train,
            callbacks=stopping_condition)

Parameters of the best model.

In [ ]:
display_best_parameters(clf)

Classification report for training data.

In [ ]:
print(classification_report(y_true=y_train,
                            y_pred=clf.predict(X_train),
                            digits=5,
                            zero_division=0.0))

Confusion matrix for training data.

In [ ]:
_ = plt.figure(figsize=(16, 9))

_ = confusion_matrix(clf.best_estimator_,
                     X_train,
                     y_train,
                     classes=labels2display,
                     cmap='Greens',
                     percent=True,
                     is_fitted=True)

Precision-recall curves for training data.

In [ ]:
_ = plt.figure(figsize=(16, 9))

_ = precision_recall_curve(clf.best_estimator_,
                           X_train,
                           y_train,
                           per_class=True)

Classification report for test data.

In [ ]:
print(classification_report(y_true=y_test,
                            y_pred=clf.predict(X_test),
                            digits=5,
                            zero_division=0.0))

Confusion matrix for test data.

In [ ]:
_ = plt.figure(figsize=(16, 9))

_ = confusion_matrix(clf.best_estimator_,
                     X_test,
                     y_test,
                     classes=labels2display,
                     cmap='Blues',
                     percent=True,
                     is_fitted=True)

Precision-recall curves for test data.

In [ ]:
_ = plt.figure(figsize=(16, 9))

_ = precision_recall_curve(clf.best_estimator_,
                           X_test,
                           y_test,
                           per_class=True)

Važnost atributa za test skup podataka.

In [ ]:
explainer = dx.Explainer(clf.best_estimator_,
                         X_train_sample,
                         enc.transform(y_train_sample),
                         predict_function=lambda a, b: enc.transform(a.predict(b)),
                         model_type='classification',
                         label='Variable importance measures for RandomForestClassifier',
                         verbose=False)

In [ ]:
mp_clf = explainer.model_parts(N=sample,
                               B=start,
                               type='shap_wrapper',
                               shap_explainer_type='KernelExplainer',
                               loss_function=lambda a, b: 1-balanced_accuracy_score(a, b),
                               processes=-1,
                               random_state=seed,
                               data=X_test,
                               silent=True)

In [ ]:
mp_clf.plot(max_display=n_features,
            plot_size=(16, 9),
            title='Variable importance measures for SVC')

In [ ]:
mp_clf.plot(plot_type='bar',
            max_display=n_features,
            plot_size=(16, 9),
            title='Variable importance measures for SVC')

### Extreme Gradient Boosting

Cross-validation parameters.

In [ ]:
params = {'n_estimators': Integer(4, 96, random_state=seed),
           'max_depth': Integer(1, 4, random_state=seed),
           'grow_policy': Categorical(['depthwise',
                                       'lossguide'], random_state=seed),
           'gamma': Continuous(0, 1, random_state=seed),
           'learning_rate': Continuous(0, 1, random_state=seed),
           'objective': Categorical(['reg:linear',
                                     'reg:logistic',
                                     'multi:softmax',
                                     'multi:softprob'], random_state=seed)}

Applying genetic search cross-validation algorithm.

In [ ]:
clf = GASearchCV(XGBClassifier(nthread=-1,
                               random_state=seed),
                 param_grid=params,
                 scoring='balanced_accuracy',
                 population_size=pops,
                 generations=maxiter,
                 tournament_size=tous,
                 elitism=True,
                 crossover_probability=crossover_adapter,
                 mutation_probability=mutation_adapter,
                 cv=cross_validation_generator,
                 algorithm='eaSimple',
                 n_jobs=2,
                 verbose=False)

_ = clf.fit(X_train,
            enc.transform(y_train),
            callbacks=stopping_condition)

Parameters of the best model.

In [ ]:
display_best_parameters(clf)

Classification report for training data.

In [ ]:
print(classification_report(y_true=y_train,
                            y_pred=enc.inverse_transform(clf.predict(X_train)),
                            digits=5,
                            zero_division=0.0))

Confusion matrix for training data.

In [ ]:
_ = plt.figure(figsize=(16, 9))

_ = confusion_matrix(clf.best_estimator_,
                     X_train,
                     enc.transform(y_train),
                     classes=labels2display,
                     cmap='Greens',
                     percent=True,
                     is_fitted=True)

Precision-recall curves for training data.

In [ ]:
y_scores = clf.predict_proba(X_train)
y_onehot = pd.DataFrame(ohe.transform(y_train), columns=ohe.classes_)

_ = plt.figure(figsize=(16, 9))

for i in range(y_scores.shape[1]):
    y_true = y_onehot.iloc[:, i]
    y_score = y_scores[:, i]

    precision, recall, _ = prec_rec_crve(y_true, y_score)
    ap_score = average_precision_score(y_true, y_score)
    name = f'{y_onehot.columns[i]} (AP={ap_score:.2f})'

    name = f'{y_onehot.columns[i]} (AP={ap_score:.2f})'
    plt.plot(recall, precision, label=name)
    
plt.xlabel('Recall')
plt.ylabel('Precision')

plt.legend()
plt.show()

Classification report for test data.

In [ ]:
print(classification_report(y_true=y_test,
                            y_pred=enc.inverse_transform(clf.predict(X_test)),
                            digits=5,
                            zero_division=0.0))

Confusion matrix for test data.

In [ ]:
_ = plt.figure(figsize=(16, 9))

_ = confusion_matrix(clf.best_estimator_,
                     X_test,
                     enc.transform(y_test),
                     classes=labels2display,
                     cmap='Blues',
                     percent=True,
                     is_fitted=True)

Precision-recall curves for test data.

In [ ]:
y_scores = clf.predict_proba(X_test)
y_onehot = pd.DataFrame(ohe.transform(y_test), columns=ohe.classes_)

_ = plt.figure(figsize=(16, 9))

for i in range(y_scores.shape[1]):
    y_true = y_onehot.iloc[:, i]
    y_score = y_scores[:, i]

    precision, recall, _ = prec_rec_crve(y_true, y_score)
    ap_score = average_precision_score(y_true, y_score)
    name = f'{y_onehot.columns[i]} (AP={ap_score:.2f})'

    name = f'{y_onehot.columns[i]} (AP={ap_score:.2f})'
    plt.plot(recall, precision, label=name)
    
plt.xlabel('Recall')
plt.ylabel('Precision')

plt.legend()
plt.show()

Važnost atributa za test skup podataka.

In [ ]:
explainer = dx.Explainer(clf.best_estimator_,
                         X_train_sample,
                         enc.transform(y_train_sample),
                         model_type='classification',
                         label='Variable importance measures for XGBClassifier',
                         verbose=False)

In [ ]:
mp_clf = explainer.model_parts(N=sample,
                               B=start,
                               type='shap_wrapper',
                               shap_explainer_type='KernelExplainer',
                               loss_function=lambda a, b: 1-balanced_accuracy_score(a, b),
                               processes=-1,
                               random_state=seed,
                               data=X_test,
                               silent=True)

In [ ]:
mp_clf.plot(max_display=n_features,
            plot_size=(16, 9),
            title='Variable importance measures for XGBClassifier')

In [ ]:
mp_clf.plot(plot_type='bar',
            max_display=n_features,
            plot_size=(16, 9),
            title='Variable importance measures for XGBClassifier')

### Linear Support Vector Machine

Parameters for cross-validation.

In [ ]:
params = {'C': Continuous(0, 1e3, random_state=seed),
          'intercept_scaling': Continuous(0, 1e3, random_state=seed)}

Applying genetic search cross-validation algorithm.

In [ ]:
clf = GASearchCV(LinearSVC(dual='auto',
                           class_weight='balanced',
                           tol=t_stop,
                           max_iter=maxiter,
                           random_state=seed),
                 param_grid=params,
                 scoring='balanced_accuracy',
                 population_size=pops,
                 generations=maxiter,
                 tournament_size=tous,
                 elitism=True,
                 crossover_probability=crossover_adapter,
                 mutation_probability=mutation_adapter,
                 cv=cross_validation_generator,
                 algorithm='eaSimple',
                 n_jobs=-1,
                 verbose=False)

_ = clf.fit(X_train,
            y_train,
            callbacks=stopping_condition)

Parameters for the best model.

In [ ]:
display_best_parameters(clf)

Probability calibration with isotonic regression.

In [ ]:
clf = CalibratedClassifierCV(clf.best_estimator_,
                             method='isotonic',
                             cv='prefit')

_ = clf.fit(X_train,
            y_train)

Classification report for training data.

In [ ]:
print(classification_report(y_true=y_train,
                            y_pred=clf.predict(X_train),
                            digits=5,
                            zero_division=0.0))

Confusion matrix for training data.

In [ ]:
_ = plt.figure(figsize=(16, 9))

_ = confusion_matrix(clf,
                     X_train,
                     y_train,
                     classes=labels2display,
                     cmap='Greens',
                     percent=True,
                     is_fitted=True)

Precision-recall curves for training data.

In [ ]:
y_scores = clf.predict_proba(X_train)
y_onehot = pd.DataFrame(ohe.transform(y_train), columns=ohe.classes_)

_ = plt.figure(figsize=(16, 9))

for i in range(y_scores.shape[1]):
    y_true = y_onehot.iloc[:, i]
    y_score = y_scores[:, i]

    precision, recall, _ = prec_rec_crve(y_true, y_score)
    ap_score = average_precision_score(y_true, y_score)
    name = f'{y_onehot.columns[i]} (AP={ap_score:.2f})'

    name = f'{y_onehot.columns[i]} (AP={ap_score:.2f})'
    plt.plot(recall, precision, label=name)
    
plt.xlabel('Recall')
plt.ylabel('Precision')

plt.legend()
plt.show()

Classification report for test data.

In [ ]:
print(classification_report(y_true=y_test,
                            y_pred=clf.predict(X_test),
                            digits=5,
                            zero_division=0.0))

Confusion matrix for test data.

In [ ]:
_ = plt.figure(figsize=(16, 9))

_ = confusion_matrix(clf,
                     X_test,
                     y_test,
                     classes=labels2display,
                     cmap='Blues',
                     percent=True,
                     is_fitted=True)

Precision-recall curves for test data.

In [ ]:
y_scores = clf.predict_proba(X_test)
y_onehot = pd.DataFrame(ohe.transform(y_test), columns=ohe.classes_)

_ = plt.figure(figsize=(16, 9))

for i in range(y_scores.shape[1]):
    y_true = y_onehot.iloc[:, i]
    y_score = y_scores[:, i]

    precision, recall, _ = prec_rec_crve(y_true, y_score)
    ap_score = average_precision_score(y_true, y_score)
    name = f'{y_onehot.columns[i]} (AP={ap_score:.2f})'

    name = f'{y_onehot.columns[i]} (AP={ap_score:.2f})'
    plt.plot(recall, precision, label=name)
    
plt.xlabel('Recall')
plt.ylabel('Precision')

plt.legend()
plt.show()

Variable importances.

In [ ]:
explainer = dx.Explainer(clf,
                         X_train_sample,
                         enc.transform(y_train_sample),
                         predict_function=lambda a, b: enc.transform(a.predict(b)),
                         model_type='classification',
                         label='Variable importance measures for LinearSVC',
                         verbose=False)

In [ ]:
mp_clf = explainer.model_parts(N=sample,
                               B=start,
                               type='shap_wrapper',
                               shap_explainer_type='KernelExplainer',
                               loss_function=lambda a, b: 1-balanced_accuracy_score(a, b),
                               processes=-1,
                               random_state=seed,
                               data=X_test,
                               silent=True)

In [ ]:
mp_clf.plot(max_display=n_features,
            plot_size=(16, 9),
            title='Variable importance measures for LinearSVC')

In [ ]:
mp_clf.plot(plot_type='bar',
            max_display=n_features,
            plot_size=(16, 9),
            title='Variable importance measures for LinearSVC')

### Artificial Neural Networks

Wrapper library *scikeras* provides *scikit-learn* high-level API for *tensorflow*/*keras* underlying library.

In [ ]:
def get_model(hidden_layer_dim,
              dropout_rate,
              activation_function,
              meta):
    model = Sequential()
    model.add(Dense(meta['n_features_in_'],
                    input_shape=meta['X_shape_'][1:],
                    activation=activation_function))
    model.add(Dropout(dropout_rate,
                      seed=seed))
    model.add(Dense(hidden_layer_dim,
                    activation=activation_function))
    model.add(Dropout(dropout_rate,
                      seed=seed))
    model.add(Dense(hidden_layer_dim,
                    activation=activation_function))
    model.add(Dropout(dropout_rate,
                      seed=seed))
    model.add(Dense(hidden_layer_dim,
                    activation=activation_function))
    model.add(Dropout(dropout_rate,
                      seed=seed))
    model.add(Dense(hidden_layer_dim,
                    activation=activation_function))
    model.add(Dropout(dropout_rate,
                      seed=seed))
    model.add(Dense(meta['n_classes_'],
                    activation='softmax'))
    
    return model

Parameters for the cross-valudation.

In [ ]:
params = {'loss': ['categorical_hinge',
                   'categorical_crossentropy'],
          'activation_function': ['relu',
                                  'sigmoid']}

Applying the cross-validation algorithm.

In [ ]:
clf = GridSearchCV(KerasClassifier(get_model,
                                   optimizer='adam',
                                   dropout_rate=0.2,
                                   hidden_layer_dim=757,
                                   activation_function='relu',
                                   # class_weight='balanced', # causes warnings
                                   metrics=Recall,
                                   epochs=maxiter,
                                   callbacks=early_stopping,
                                   validation_split=0.2,
                                   batch_size=-1,
                                   verbose=0,
                                   random_state=seed),
                   param_grid=params,
                   scoring=lambda a, b: balanced_accuracy_score(ohe.inverse_transform(a),
                                                                ohe.inverse_transform(b)),
                   cv=cross_validation_generator,
                   n_jobs=2)

_ = clf.fit(X_train,
            ohe.transform(y_train))

Parameters for the best model.

In [ ]:
display_best_parameters(clf)

Classification report for training data.

In [ ]:
print(classification_report(y_true=y_train,
                            y_pred=ohe.inverse_transform(clf.predict(X_train)),
                            digits=5,
                            zero_division=0.0))

Confusion matrix for training data.

In [ ]:
_ = plot_confusion_matrix(conf_mat=conf_matrx(y_target=y_train,
                                              y_predicted=ohe.inverse_transform(clf.predict(X_train))),
                          figsize=(16, 9),
                          cmap='Greens',
                          show_absolute=False,
                          show_normed=True,
                          class_names=labels2display)

Precision-recall curves for training data.

In [ ]:
y_scores = clf.predict_proba(X_train)
y_onehot = pd.DataFrame(ohe.transform(y_train), columns=ohe.classes_)

_ = plt.figure(figsize=(16, 9))

for i in range(y_scores.shape[1]):
    y_true = y_onehot.iloc[:, i]
    y_score = y_scores[:, i]

    precision, recall, _ = prec_rec_crve(y_true, y_score)
    ap_score = average_precision_score(y_true, y_score)
    name = f'{y_onehot.columns[i]} (AP={ap_score:.2f})'

    name = f'{y_onehot.columns[i]} (AP={ap_score:.2f})'
    plt.plot(recall, precision, label=name)
    
plt.xlabel('Recall')
plt.ylabel('Precision')

plt.legend()
plt.show()

Classification report for test data.

In [ ]:
print(classification_report(y_true=y_test,
                            y_pred=ohe.inverse_transform(clf.predict(X_test)),
                            digits=5,
                            zero_division=0.0))

Confusion matrix for test data.

In [ ]:
_ = plot_confusion_matrix(conf_mat=conf_matrx(y_target=y_test,
                                              y_predicted=ohe.inverse_transform(clf.predict(X_test))),
                          figsize=(16, 9),
                          cmap='Blues',
                          show_absolute=False,
                          show_normed=True,
                          class_names=labels2display)

Precision-recall curves for test data.

In [ ]:
y_scores = clf.predict_proba(X_test)
y_onehot = pd.DataFrame(ohe.transform(y_test), columns=ohe.classes_)

_ = plt.figure(figsize=(16, 9))

for i in range(y_scores.shape[1]):
    y_true = y_onehot.iloc[:, i]
    y_score = y_scores[:, i]

    precision, recall, _ = prec_rec_crve(y_true, y_score)
    ap_score = average_precision_score(y_true, y_score)
    name = f'{y_onehot.columns[i]} (AP={ap_score:.2f})'

    name = f'{y_onehot.columns[i]} (AP={ap_score:.2f})'
    plt.plot(recall, precision, label=name)
    
plt.xlabel('Recall')
plt.ylabel('Precision')

plt.legend()
plt.show()

Feature importances.

In [ ]:
explainer = dx.Explainer(clf.best_estimator_,
                         X_train_sample,
                         enc.transform(y_train_sample),
                         predict_function=lambda a, b: enc.transform(ohe.inverse_transform(a.predict(b))),
                         model_type='classification',
                         label='Variable importance measures for KerasClassifier',
                         verbose=False)

In [ ]:
mp_clf = explainer.model_parts(N=sample,
                               B=start,
                               type='shap_wrapper',
                               shap_explainer_type='KernelExplainer',
                               loss_function=lambda a, b: 1-balanced_accuracy_score(a, b),
                               processes=-1,
                               random_state=seed,
                               data=X_test,
                               silent=True)

In [ ]:
mp_clf.plot(max_display=n_features,
            plot_size=(16, 9),
            title='Variable importance measures for KerasClassifier')

In [ ]:
mp_clf.plot(plot_type='bar',
            max_display=n_features,
            plot_size=(16, 9),
            title='Variable importance measures for KerasClassifier')

### Aproximate k-Nearest Neighbors

A more efficient implementation of k-Nearest Neighbors search than the one given in *scikit-learn*.
Initially, I tried various implementations based on HSNW search, but I could not find a library that builds indexer deterministically.
*Annoy* indexer might be a bit slower, but at least every time I retrain the classifier, with the same set of parameters, I get the same result.

In [ ]:
class AnnoyKNN:
    _estimator_type = "classifier"
    
    def __init__(self,
                 weighted=False,
                 metric='manhattan',
                 k=5,
                 n_trees=128,
                 n_jobs=1,
                 random_state=None):
        assert isinstance(weighted, bool), 'weighted has to be bool'
        assert metric in ('angular',
                          'euclidean',
                          'manhattan',
                          'hamming',
                          'dot'), 'metric can be "angular", "euclidean", "manhattan", "hamming", or "dot"'
        assert isinstance(k, int), 'k has to be int'
        assert isinstance(n_trees, int), 'n_trees has to be int'
        assert isinstance(n_jobs, int), 'n_jobs has to be int'
        assert isinstance(random_state, (int, type(None))), 'random_state has to be int or None'
        
        self.index = None
        self.X = pd.DataFrame()
        self.y = None
        self.classes_ = None
        self.k = k
        self.metric = metric
        self.n_trees = n_trees
        self.n_jobs = n_jobs
        self.weighted = weighted
        self.random_state = random_state

    def __sklearn_clone__(self):
        clone = AnnoyKNN(self.weighted,
                         self.metric,
                         self.k,
                         self.n_trees,
                         self.n_jobs,
                         self.random_state)
        clone.X = self.X
        clone.index = self.index
        
        return clone
    
    def get_params(self, deep=True):
        return {'index': self.index,
                'y': self.y,
                'k': self.k,
                'classes_': self.classes_,
                'metric': self.metric,
                'n_trees': self.n_trees,
                'n_jobs': self.n_jobs,
                'weighted': self.weighted,
                'random_state': self.random_state}
    
    def set_params(self, **params):
        for key, val in params.items():
            if key in ('metric',
                       'n_trees',
                       'random_state'):
                self.index = None
            self.__dict__[key] = val

        assert isinstance(self.weighted, bool), 'weighted has to be bool'
        assert self.metric in ('angular',
                               'euclidean',
                               'manhattan',
                               'hamming',
                               'dot'), 'metric can be "angular", "euclidean", "manhattan", "hamming", or "dot"'
        assert isinstance(self.k, int), 'k has to be int'
        assert isinstance(self.n_trees, int), 'n_trees has to be int'
        assert isinstance(self.n_jobs, int), 'n_jobs has to be int'
        assert isinstance(self.random_state, (int, type(None))), 'random_state has to be int or None'
        assert isinstance(self.X, (pd.DataFrame, np.ndarray)), 'X has to be pandas DataFrame or numpy array object'
        self.X = self.X if isinstance(self.X, pd.DataFrame) else pd.DataFrame(self.X)
        
        return self

    def fit(self, X, y):
        assert isinstance(X, (pd.DataFrame, np.ndarray)), 'X has to be pandas DataFrame or numpy array object'
        X = X if isinstance(X, pd.DataFrame) else pd.DataFrame(X)
        
        if self.index is None or not X.equals(self.X):
            self.X = X
            
            self.index = AnnoyIndex(X.shape[1], self.metric)
            self.index.set_seed(self.random_state)
            
            for i, v in enumerate(X.to_numpy()):
                self.index.add_item(i, v)
            self.index.build(self.n_trees, self.n_jobs)
            
        assert isinstance(y, (pd.Series, np.ndarray)), 'y has to be pandas Series object'
        self.y = y.to_numpy() if isinstance(y, pd.Series) else y
        self.classes_ = np.unique(self.y)
        
        return self

    def score(self, X, y, sample_weight=None):
        assert self.index is not None, 'first to fit the model you have'
        assert isinstance(X, (pd.DataFrame, np.ndarray)), 'X has to be pandas DataFrame or numpy array object'
        assert isinstance(y, (pd.Series, np.ndarray)), 'y has to be pandas Series or numpy array object'
        
        return balanced_accuracy_score(y_true=y,
                                       y_pred=self.predict(X),
                                       sample_weight=sample_weight)

    def __get_weighted_votes(self, X):
        indices = np.empty((X.shape[0], self.k),
                           dtype=np.int32)
        distances = np.empty((X.shape[0], self.k),
                             dtype=np.float64)
        
        # Annoy is annoying for not having batch query ...
        for i, v in enumerate(X.to_numpy()):
            tmp_i, tmp_d = self.index.get_nns_by_vector(v,
                                                        self.k,
                                                        include_distances=self.weighted)
            indices[i] = np.array(tmp_i,
                                  dtype=np.int32)
            distances[i] = np.array(tmp_d,
                                    dtype=np.float64)
        
        vote = self.y[indices]
        weight = np.reciprocal(distances,
                               where=distances!=0,
                               out=np.zeros_like(distances))
        weighted_votes = [(vote[a], weight[a]) for a in range(indices.shape[0])]
        
        return weighted_votes
    
    def __get_unweighted_votes(self, X):        
        indices = np.empty((X.shape[0], self.k),
                           dtype=np.int32)
        
        # Annoy is annoying for not having batch query ...
        for i, v in enumerate(X.to_numpy()):
            tmp_i = self.index.get_nns_by_vector(v,
                                                 self.k,
                                                 include_distances=self.weighted)
            indices[i] = np.array(tmp_i,
                                  dtype=np.int32)
        
        unweighted_votes = self.y[indices]
        
        return unweighted_votes

    def predict(self, X):
        assert isinstance(X, (pd.DataFrame, np.ndarray)), 'X has to be pandas DataFrame or numpy array object'
        X = X if isinstance(X, pd.DataFrame) else pd.DataFrame(X)
        predictions = None
        
        if self.weighted:
            votes = self.__get_weighted_votes(X)
            
            predictions = np.array([np.argmax(np.bincount(a,
                                                          weights=b)) for a, b in votes],
                                   dtype=np.int32)
        else:
            votes = self.__get_unweighted_votes(X)
            
            predictions = np.array([np.argmax(np.bincount(a)) for a in votes],
                                   dtype=np.int32)
            
        return pd.Series(predictions)
    
    def predict_proba(self, X):
        assert isinstance(X, (pd.DataFrame, np.ndarray)), 'X has to be pandas DataFrame or numpy array object'
        X = X if isinstance(X, pd.DataFrame) else pd.DataFrame(X)
        probabilities = None
        
        if self.weighted:
            votes = self.__get_weighted_votes(X)
            
            numerator = np.array([np.bincount(a,
                                              weights=b,
                                              minlength=self.classes_.shape[0]) for a, b in votes])
            denominator = np.array([a.sum() for a in numerator])
            probabilities = np.array([a / b for a, b in zip(numerator, denominator)])
        else:
            votes = self.__get_unweighted_votes(X)

            numerator = np.array([np.bincount(a,
                                              minlength=self.classes_.shape[0]) for a in votes])
            denominator = self.k
            probabilities = numerator / denominator
            
        return probabilities

Parameters for cross-validation.

In [ ]:
params = [{'k': [5, 13, 23, 31],
          'weighted': [True, False]}]

Applying genetic search cross-validation algorithm.

In [ ]:
clf = GridSearchCV(AnnoyKNN(n_jobs=-1,
                            random_state=seed),
                   param_grid=params,
                   scoring='balanced_accuracy',
                   cv=cross_validation_generator)

_ = clf.fit(X_train,
            enc.transform(y_train))

Parameters of the best model.

In [ ]:
display_best_parameters(clf)

Classification report for training data.

In [ ]:
print(classification_report(y_true=y_train,
                            y_pred=enc.inverse_transform(clf.predict(X_train)),
                            digits=5,
                            zero_division=0.0))

Confusion matrix for training data.

In [ ]:
_ = plt.figure(figsize=(16, 9))

_ = confusion_matrix(classifier(clf.best_estimator_),
                     X_train,
                     enc.transform(y_train),
                     classes=labels2display,
                     cmap='Greens',
                     percent=True)

Precision-recall curves for training data.

In [ ]:
_ = plt.figure(figsize=(16, 9))

_ = precision_recall_curve(classifier(clf.best_estimator_),
                           X_train,
                           enc.transform(y_train),
                           classes=labels2display,
                           per_class=True)

Classification report for test data.

In [ ]:
print(classification_report(y_true=y_test,
                            y_pred=enc.inverse_transform(clf.predict(X_test)),
                            digits=5,
                            zero_division=0.0))

Confusion matrix for test data.

In [ ]:
_ = plt.figure(figsize=(16, 9))

_ = confusion_matrix(classifier(clf.best_estimator_),
                     X_test,
                     enc.transform(y_test),
                     classes=labels2display,
                     cmap='Blues',
                     percent=True,
                     is_fitted=True)

Precision-recall curves for test data.

In [ ]:
_ = plt.figure(figsize=(16, 9))

_ = precision_recall_curve(classifier(clf.best_estimator_),
                           X_test,
                           enc.transform(y_test),
                           classes=labels2display,
                           per_class=True)

Feature importances.
Since SHAP library dislikes my implementation of AnnoyKNN, and since it's so well documented that it makes debugging impossible, instead of using SHAP values, simple permutation importances will be calculated.

In [ ]:
explainer = dx.Explainer(clf.best_estimator_,
                         X_test,
                         enc.transform(y_test),
                         precalculate=False,
                         label='Variable importance measures for AnnoyKNN',
                         verbose=False)

In [ ]:
mp_clf = explainer.model_parts(keep_raw_permutations=False,
                               random_state=seed)

In [ ]:
df = mp_clf.result.copy()
df = df.set_index('variable')
df = df.drop(index=['_baseline_', '_full_model_'])

df['dropout_loss'] = df['dropout_loss'] - df['dropout_loss'].min()

In [ ]:
_ = plt.figure(figsize=(16, 9))

_ = sns.barplot(data=df.nlargest(n_features, 'dropout_loss'),
                y='variable',
                x='dropout_loss',
                orient='h',
                color='royalblue')

plt.show()

## Conclusion

After loading the data, features containing coding sequences and protein types have been extracted.
Then, we cleaned up the data by dropping the sequences with inappropriate number of nucleotides, with ambiguities, or belong to the classes represented by too few instances for reliable generalization.

The preparation phase has been completed by calculating codon usage frequencies and splitting the data into train and test sets.
A couple of other utilities have been prepared, before moving on to visualization phase.

During the visualization phase, we first looked at some general statistics.
That is, distribution of instances in different classes and features correlation test with target variable.
Then we looked at parallel coordinates and radar chart of a sampled data, and finally we looked at *t-SNE* projection of the training dataset.
The classes are extremely imbalanced, but they seem to be separable.
Then, we moved on to classification.

It appears that the data is linearly separable, as the linear-like classifiers such as *LinearSVC* and *DecisionTreeClassifier* achieved excellent results.
Even my custom built *AnnoyKNN* classifier, based on *Annoy* indexer, favored fewer neighbors and ignored distances, which implies that it prefers more linear separation of the data.

*ComplementaryNB* struggled with imbalanced data.
It simply failed to recognize the underrepresented classes.

The best results have been achieved with ensemble methods based on decision trees.
That doesn't come as a surprise, as even a single decision tree has achieved significant result.

This version of *KerasClassifier* appears to be broken.
In conjunction with *GASearchCV* it fails to report classification score, which is necessary for triggering early stopping callback.
Instead it returned NaN values.
When used with ordinary *GridSearchCV* it triggerd a number of warnings for variables that are even not accessible from *scikeras* API.

This version of *GASearchCV* does not parallelize at all, despite having n_jobs parameter.
If it has to evaluate *n* models during an iteration, and it has *n* cores available, I would expect that it would train each model in a separate process.
Instead, the models are trained one at a time, which is a serious flaw of this implementation.

Despite imbalance, the data classifies really well.
The majority of classifiers have achieved nearly $100\%$ accuracy, which is very rare.
Overall, out of seven reained models, six demonstrated excellent classification performance.